# Assignment 1: Transformations and Representations

Roll number: \<Roll number here\>


# Instructions

- Code must be written in Python in Jupyter Notebooks. We highly recommend using anaconda distribution or at the minimum, virtual environments for this assignment. 
- Save all your results in ```results/<question_number>/<sub_topic_number>/```
- For this assignment, you will be using Open3D extensively. Refer to [Open3D Documentation](http://www.open3d.org/docs/release/): you can use the in-built methods and **unless explicitly mentioned**, don't need to code from scratch for this assignment. 
- Make sure your code is modular since you may need to reuse parts for future assignments.
- Make sure any extra files you that you need to submit, place it in *'results'* folder.
- Answer the descriptive questions in your own words with context & clarity. Do not copy answers from online resources or lecture notes.
- The **deadline** for this assignment is on **23/08/2022 at 11:55pm**. Please note that there will be no extensions.
- Plagiarism is **strictly prohibited**.


# Submission Instructions

1. Make sure your code runs without any errors after reinitializing the kernel and removing all saved variables.
2. After completing your code and saving your results, zip the folder with name as ``<roll_number>_MR2022_<assignment_number>.zip``

## 1. 3D Data and Open3D

1. Please find mesh files in **data/Q1** folder. Using these mesh files and your own creativity/visualisation, create a "Table" **pointcloud** scene. The table scene should be realistic, scaled appropiately. Use all the meshes given in the folder and treat them as objects kept on the table. 

    You are expected to perform different functions on the individual mesh files: first convert the mesh files to pointclouds and on each pointcloud perform operations such as scaling, rotation, translation. Next, visualize them together. The visualization should represent a pointcloud of a realistic table scene. Save the scene as **.pcd** file. 

    **Please do not copy as we may use your contribution to create a table top dataset.**

    Refer below image for example of a table-top scene:

    <img src="img/1.jpeg"  width="500" >
<br>
<br>

2. Use the final table scene pointcloud obtained from part 1. 
    - Use Open3D to generate partial pointclouds from different camera views (at least 4 views). This means, that you need to crop or capture the points in the pointcloud that are visible only from a given viewpoint. 
    - Using these partial pointclouds, you are now expected to generate the full scene pointcloud back by registering the pointclouds to a global frame. Save the partial and reconstructed pointclouds in different files. 
    - **[ BONUS ]** Finally, compute the error using "Chamfer's Distance (CD)" between the ground truth scene pointcloud and the reconstructed pointcloud. Perform an analysis: 
      1. Why is the CD not 0?
      2. How does the CD change as the number of viewpoints increase / decrease? 
      3. Can we optimize the viewpoints (by hit and trial) such that the CD reduces?

Refer the following link for solving Q1:
- Hidden-Point-Removal Open3D API: http://www.open3d.org/docs/latest/tutorial/Basic/pointcloud.html#Hidden-point-removal

- Chamfer's Distance:  https://pytorch3d.readthedocs.io/en/latest/modules/loss.html



### References for this question:
* how to open obj file and save as ply file: http://www.open3d.org/docs/0.9.0/tutorial/Basic/file_io.html
* how to convert obj or ply file to pcd file via sampling : http://www.open3d.org/docs/release/tutorial/geometry/mesh.html

In [7]:
import numpy as np
import matplotlib
import open3d as o3d
import os
from scipy import stats as st

data_path ="data/Q1"
results_path ="results/Q1/pointclouds"
table_path = "results/Q1/pointclouds/table.pcd"
pcd_list = []

def configure_environment():
    o3d.utility.set_verbosity_level(o3d.utility.VerbosityLevel.Error)
    np.set_printoptions(suppress=True)

def read_mesh_write_pointcloud(data_path,results_path):
    for file in os.listdir(data_path):
        print(f"{file}")
        mesh = o3d.io.read_triangle_mesh(f"{data_path}/{file}")
        #print(mesh)  
        pointcloud = mesh.sample_points_poisson_disk(number_of_points=1000)
        o3d.io.write_point_cloud(f"{results_path}/{os.path.splitext(file)[0]}.pcd", pointcloud)
        
def read_table_bounds(pcd_list, path):
    pcd = o3d.io.read_point_cloud(path)
    R = pcd.get_rotation_matrix_from_xyz((-np.pi/2, 0, -np.pi/2))
    pcd = pcd.rotate(R, center=pcd.get_center())
    pcd.translate((0,0,0), relative=False)
            
    #segment plane        
    plane_model, inliers = pcd.segment_plane(distance_threshold = 0.01, ransac_n = 3, num_iterations =1000)
    [a, b, c, d] = plane_model
            
    inlier_cloud =pcd.select_by_index(inliers)
    inlier_cloud.paint_uniform_color([1.0, 0, 0])
    outlier_cloud = pcd.select_by_index(inliers,invert=True)
    outlier_cloud.paint_uniform_color([1, 0.2, 0.2])
            
    #get min and max x of the inlier cloud
    min_x = np.min(np.array(inlier_cloud.points), axis=0)[0]
    max_x = np.max(np.array(inlier_cloud.points), axis=0)[0]
    #get min and max z of the inlier cloud        
    min_z = np.min(np.array(inlier_cloud.points), axis=0)[2]
    max_z = np.max(np.array(inlier_cloud.points), axis=0)[2]
            
            
    pcd_list.append(inlier_cloud)
    pcd_list.append(outlier_cloud)
    
    return (min_x, max_x, min_z, max_z)

def read_obj_bounds(pcd):
    #get min and max x of the inlier cloud
    min_x = np.min(np.array(pcd.points), axis=0)[0]
    max_x = np.max(np.array(pcd.points), axis=0)[0]
    #get min and max z of the inlier cloud        
    min_z = np.min(np.array(pcd.points), axis=0)[2]
    max_z = np.max(np.array(pcd.points), axis=0)[2]
    

def read_table_level(path):
    pcd = o3d.io.read_point_cloud(path)
    R = pcd.get_rotation_matrix_from_xyz((-np.pi/2, 0, -np.pi/2))
    pcd = pcd.rotate(R, center=pcd.get_center())
    pcd.translate((0,0,0), relative=False)
            
    #segment plane        
    plane_model, inliers = pcd.segment_plane(distance_threshold = 0.01, ransac_n = 3, num_iterations =1000)
    [a, b, c, d] = plane_model
            
    inlier_cloud =pcd.select_by_index(inliers)
    bias = 1
    table_surface = np.max(np.array(inlier_cloud.points), axis=0)[1]+bias
    return table_surface    

def sample_random_pos(table_bounds):
    bias = 3
    posx = np.random.randint(low=table_bounds[0]+bias, high=table_bounds[1]-bias)
    posz = np.random.randint(low=table_bounds[2]+bias, high=table_bounds[3]-bias)
    return (posx,posz)
   
    
def viz_pcd(results_path,pcd_list, table_bounds, table_level):
    #create a coordinate frame
    obj_number = 0
    world_coordinate_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(size=20, origin=[0, 0, 0])
    #append the world coordinate frame to the list
    pcd_list.append(world_coordinate_frame)
    #iterate over all the files in the directory
    for file in os.listdir(results_path):
        obj_number+= 1
        #print(f"{file}")
        #read point clouds
        pcd = o3d.io.read_point_cloud(f"{results_path}/{file}")
        #if the point cloud is not table scale the parts
        if(os.path.splitext(file)[0] != "table"):
            posx, posz = sample_random_pos(table_bounds)
            pcd = pcd.scale(7, center=pcd.get_center())
            pcd.translate((posx,table_level, posz), relative=False)
            print(f"{os.path.splitext(file)[0]} center:{pcd.get_center()}")
            pcd.paint_uniform_color([0.111*obj_number, .143*obj_number, 1-0.023*obj_number])
            pcd_list.append(pcd)
        else:
            print(f"{os.path.splitext(file)[0]} center:{pcd.get_center()}")     
            
    o3d.visualization.draw_geometries(pcd_list)
    

In [8]:
configure_environment()
read_mesh_write_pointcloud(data_path,results_path)
table_bounds = np.array(read_table_bounds(pcd_list, table_path), dtype=int)
table_level = read_table_level(table_path)
print(table_level)
viz_pcd(results_path,pcd_list, table_bounds, table_level)

boat.obj
trashcan.obj
plane.obj
laptop.obj
table.obj
car.obj
4.517631504846264
trashcan center:[7.        4.5176315 5.       ]
car center:[-14.          4.5176315  -4.       ]
table center:[89.11841909 90.74375466 25.6229687 ]
laptop center:[ 4.         4.5176315 -3.       ]
boat center:[-12.          4.5176315 -10.       ]
plane center:[-11.          4.5176315   9.       ]


## 2. Euler Angles, Rotation Matrices, and Quaternions
1. Write a function (do not use inbuilt libraries for this question):
    - that returns a rotation matrix given the angles $\alpha$, $\beta$, and $\gamma$ in radians (X-Y-Z).
    - to convert a rotation matrix to quaternion and vice versa. 

2. What is a Gimbal lock? Suppose an airplane increases its pitch from $0°$ to $90°$. 

    - Let $R_{gmb\beta}$ be the rotation matrix for $\beta=90°$. Find $R_{gmb\beta}$.
    - Consider the point $p = [0, 1, 0]ᵀ $ on the pitched airplane, i.e. the tip of the wing. Does there exist any $α , γ$ such that $p_{new} = R_{gmb\beta}\; p$ for:
      1. $p_{new} = [1, 0, 0]ᵀ $
      2. For some  $p_{new}$ on the XY unit circle?
      3. For some  $p_{new}$ on the YZ unit circle?
      
      Show your work for all three parts and briefly explain your reasoning. Why is $\beta=90°$  a “certain problematic value”?

    <img src="img/2.3.jpeg"  width="500" ><br>
    
    <img src="img/2.1.jpeg"  width="500" ><br>

    <img src="img/2.2.jpeg"  width="500" >
    


In [9]:
#2.1
import numpy as np

alpha = np.radians(90)
beta = np.radians(90)
gamma = np.radians(0) 

#method to calculate the rotation matrix given euler angles of rotation
def R(angles):
    phi = angles[0]
    theta = angles[1]
    psi = angles[2]
    R_x = np.array( [ [ 1 , 0 , 0 ] ,
                   [ 0 , np.cos(phi) , -np.sin(phi) ]  , 
                   [0 , np.sin(phi) , np.cos(phi)]])
    
    R_y = np.array( [ [ np.cos(theta) , 0 , np.sin(theta) ] ,
                   [ 0 ,1 ,0 ] ,
                   [ -np.sin(theta) , 0 , np.cos(theta)]])
    R_z = np.array( [ [  np.cos(psi) , -np.sin(psi) , 0 ] , 
                   [np.sin(psi) , np.cos(psi) , 0 ] ,
                   [0 ,0, 1]]  )
    Rm = R_z@R_y@R_x

    return Rm

def R2Q(Rm):
    M = np.array(Rm)
    tr = np.trace(M)
    if(tr > 0):
        sq = np.sqrt(tr + 1)*2
        qw = 0.25*sq
        qx = (M[2][1] - M[1][2])/sq
        qy = (M[0][2] - M[2][0])/sq
        qz = (M[1][0] - M[0][1])/sq
        
    elif(M[0][0] > M[1][1] and M[0][0] > M[2][2]):
        sq = np.sqrt(1 + M[0][0] - M[1][1] - M[2][2])*2
        qw = (M[2][1] - M[1][2])/sq
        qx = 0.25*sq
        qy = (M[0][1] + M[1][0])/sq
        qz = (M[0][2] + M[2][0])/sq
        
    elif(M[1][1] > M[2][2]):
        sq = np.sqrt(1 + M[1][1] - M[0][0] - M[2][2])*2
        qw = (M[0][2] - M[2][0])/sq
        qx = (M[0][1] + M[1][0])/sq
        qy = 0.25*sq
        qz = (M[1][2] + M[2][1])/sq
        
    else:
        sq = np.sqrt(1 + M[2][2] - M[0][0] - M[1][1])*2
        qw = (M[1][0] - M[0][1])/sq
        qx = (M[0][2] + M[2][0])/sq
        qy = (M[1][2] + M[2][1])/sq
        qz = 0.25*sq

    q = np.array([qw,qx,qy,qz])
    return q

def Q2R(q):
    norm_reciprocal = 1/np.sqrt(q[0]*q[0] + q[1]*q[1] + q[2]*q[2] + q[3]*q[3]) #Normalize
    qw = q[0]*norm_reciprocal
    qx = q[1]*norm_reciprocal
    qy = q[2]*norm_reciprocal
    qz = q[3]*norm_reciprocal
    
    Rm = np.array([[2*(qw*qw + qx * qx)-1, 2*(qx*qy - qw*qz), 2*(qx*qz + qw*qy)],
                           [2*(qx*qy + qw*qz),  2*(qw*qw + qy*qy) -1, 2*(qy*qz - qw*qx)],
                           [2*(qx*qz - qw*qy), 2*(qy*qz + qw*qx), 2*(qw*qw + qz*qz)-1]])
                            
    return Rm



angles = np.array( [alpha, beta, gamma]) # X, Y, Z
print(f"The entered euler angles are[alpha,beta,gamma]: {angles}")
Rotation_Matrix=R(angles)
print(f"Rotation_Matrix:{Rotation_Matrix}")

q = R2Q(Rotation_Matrix)
print(f"quaternion: {q}")

Rotation_Matrix_Reobtained = Q2R(q)

print(f"Rotation_matrix_reobtained: {Rotation_Matrix_Reobtained}")


The entered euler angles are[alpha,beta,gamma]: [1.57079633 1.57079633 0.        ]
Rotation_Matrix:[[ 0.  1.  0.]
 [ 0.  0. -1.]
 [-1.  0.  0.]]
quaternion: [ 0.5  0.5  0.5 -0.5]
Rotation_matrix_reobtained: [[ 0.  1.  0.]
 [ 0.  0. -1.]
 [-1.  0.  0.]]


It can be seen that the rotation matrix obtained from the euler angles and the rotation matrix obtained reobtained from the quaternions are equal and consistent.

### Q2
What is Gimbal lock?
When two rotational axis are aligned with each other or parallel to each other, it is called a gimbal lock. In a gimbal lock, 1 DoF is lost.

### Q2.2 1st bullet

In [10]:
alpha = 0
beta = np.radians(90)
gamma = 0

angles = np.array([alpha,beta,gamma])
Rotation_Matrix=R(angles)
print(f"Rotation_Matrix:{Rotation_Matrix}")

Rotation_Matrix:[[ 0.  0.  1.]
 [ 0.  1.  0.]
 [-1.  0.  0.]]


### Q2.2 2nd bullet
* There exists two gamma and infinitely many alpha, considering that range of rotation is [-pi, pi] rads around each axis, for which the value of Pnew = [1,0,0]


## 3. Transformations and Homogeneous Coordinates

1. Watch this [video](https://www.youtube.com/watch?v=PvEl63t-opM) to briefly understand homogeneous coordinates. 
    1. What are points at infinity? What type of transformation can you apply to transform a point from infinity to a point that is not at infinity? 
    2. Find the vanishing point for the given images in the **data/Q3** folder. Complete function **FilterLines()** and  **GetVanishingPoint()** in the given starter code.

<br>

2. Using homogeneous coordinates we can represent different types of transformation as point transforms vs. frame transforms. Concatenation of transforms (whether you post multiply transformation matrices or pre-multiply transformation matrices) depends on the problem and how you are viewing it. Try to understand the difference between frame vs. point transformations from this [video](https://youtu.be/Za7Sdegf8m8?t=1834). We have 5 camera frames A, B, C, D and E. Given *frame* transformation $A \rightarrow B$ ,  $B \rightarrow C$ ,  $D \rightarrow C$ ,  $D \rightarrow E$. Compute *frame transformation*  $D \rightarrow E$. Also, given the co-ordinates of a point *x* in *D's* frame, what transformation is required to get *x's*  co-ordinates in *E's* frame? 

    <img src="img/3.jpeg"  width="500" >



In [ ]:
import os
import cv2
import math
import numpy as np
import matplotlib.pyplot as plt

def ReadImage(InputImagePath):
    Images = []                     # Input Images will be stored in this list.
    ImageNames = []                 # Names of input images will be stored in this list.
    
    # Checking if path is of file or folder.
    if os.path.isfile(InputImagePath):						    # If path is of file.
        InputImage = cv2.imread(InputImagePath)                 # Reading the image.
        
        # Checking if image is read.
        if InputImage is None:
            print("Image not read. Provide a correct path")
            exit()
        
        Images.append(InputImage)                               # Storing the image.
        ImageNames.append(os.path.basename(InputImagePath))     # Storing the image's name.

	# If path is of a folder contaning images.
    elif os.path.isdir(InputImagePath):
		# Getting all image's name present inside the folder.
        for ImageName in os.listdir(InputImagePath):
			# Reading images one by one.
            InputImage = cv2.imread(InputImagePath + "/" + ImageName)
			
            Images.append(InputImage)							# Storing images.
            ImageNames.append(ImageName)                        # Storing image's names.
        
    # If it is neither file nor folder(Invalid Path).
    else:
        print("\nEnter valid Image Path.\n")
        exit()

    return Images, ImageNames
        
def GetLines(Image):

    GrayImage = cv2.cvtColor(Image, cv2.COLOR_BGR2GRAY)
    BlurGrayImage = cv2.GaussianBlur(GrayImage, (5, 5), 1)
    EdgeImage = cv2.Canny(BlurGrayImage, 40, 255)
    Lines = cv2.HoughLinesP(EdgeImage, 1, np.pi / 180, 50, 10, 15)
    if Lines is None:
        print("No enough lines for vanishing point detection")
        exit(0)
    
    return Lines
    
def FilterLines(Lines):
    fil_lines = []
    for Line in Lines:
        dx = Line[0][2] - Line[0][0]
        if dx == 0:
            dx = 0.000001
        dy = Line[0][3] - Line[0][1]
        slope = np.arctan2(dy, dx)
        if np.abs(np.rad2deg(slope)) >= 2.0 and np.abs(np.rad2deg(slope)) <= 88.0:
            fil_lines.append([Line[0][0], Line[0][1], Line[0][2], Line[0][3], dy/dx])

    return fil_lines

def GetVanishingPoint(FilteredLines):
    iters = 1000
    line_pts_og = FilteredLines
    min_dist = np.inf
    vp_opt = [300, 400]
    for i in range(iters):
        line_pts = copy.deepcopy(line_pts_og)
        idx = random.sample(line_pts, 2)
        line_pts.remove(idx[0])
        line_pts.remove(idx[1])
        l1 = idx[0]
        l2 = idx[1]

        dx = l1[2] - l1[0]
        if dx == 0:
            dx = 0.00001
        dy = l1[3] - l1[1]
        m1 = dy/dx
        a1 = -m1
        b1 = 1
        c1 = l1[1] - m1*l1[0]

        dx = l2[2] - l2[0]
        if dx == 0:
            dx = 0.00001
        dy = l2[3] - l2[1]
        m2 = dy/dx
        a2 = -m2
        b2 = 1
        c2 = l2[1] -m2*l2[0]

        ix, iy = np.abs((b1*c2 - b2*c1)/(a1*b2 - a2*b1)), np.abs((c1*a2 - c2*a1)/(a1*b2 - a2*b1))

        pts = np.array(line_pts)

        a_line = -pts[:, 4]
        b_line = 1
        c_line = pts[:, 1] - pts[:, 4]*pts[:, 0]
        dist_from_ip = np.abs(a_line*pts[:, 0] + b_line*pts[:, 1] + c_line)/np.sqrt(a_line**2 + b_line**2)
        min_idx = np.argmin(dist_from_ip)
        dist = dist_from_ip[min_idx]
        if dist<min_dist:
            min_dist = dist
            vp_opt = [ix, iy]
    return vp_opt

Images, ImageNames = ReadImage("data/Q3")            # Reading all input images


for i in range(len(Images)):
    Image = Images[i]
    Lines = GetLines(Image)
    FilteredLines = FilterLines(Lines)
    VanishingPoint = GetVanishingPoint(FilteredLines)

    if VanishingPoint is None:
        print("No Vanisihing points")
        continue
    for Line in Lines:
        cv2.line(Image, (Line[0], Line[1]), (Line[2], Line[3]), (0, 255, 0), 2)
    cv2.circle(Image, (int(VanishingPoint[0]), int(VanishingPoint[1])), 10, (0, 0, 255), -1)

    plt.imshow(Image)
    plt.title("plot")
    plt.show()

## 4. LiDAR and Registration

Point clouds are a collection of points that represent a 3D shape or feature. Each point has its own set of X, Y and Z coordinates and in some cases additional attributes. A popular way to obtain this is by photogrammetry, though here we will use LiDAR data.

LiDAR is a remote sensing process which collects measurements used to create 3D models and maps of objects and environments. Using ultraviolet, visible, or near-infrared light, LiDAR gauges spatial relationships and shapes by measuring the time it takes for signals to bounce off objects and return to the scanner.

Download the data from [here](https://iiitaphyd-my.sharepoint.com/:f:/g/personal/venkata_surya_students_iiit_ac_in/EnYAMaTVIhJItzKYqtahE30BRKB6p6UfHN3TyJzvo6Mw0g?e=PegWds). It contains the LIDAR sensor output and odometry information per frame.

  The .bin files contain the 3D point cloud captured by the LIDAR in this format - x, y, z, and reflectance. 

  The odometry information is given in the `odometry.txt` file, which is a 12 element vector. Reshape each of the first 77 rows to a 3x4 matrix to obtain the pose.
    
The point cloud obtained is with respect to the LiDAR frame. The poses however, are in the camera frame. If we want to combine the point clouds from various frames, we need to bring them to the camera frame. 

1. Refer to the image below and apply the required transformation to the point cloud. 
<br>

    <img src="img/4.jpeg"  width="500" >

<br>

2. Then, register all point clouds into a common reference frame and visualise it (Open3D). It is helpful to use homogeneous coordinates to keep track of the different frames.

3. Write a function to transform the registered point cloud from the world to the $i^{th}$ camera frame, wherein $i$ is the input to the function.


